# AI LAB#3
# Hassan Imran
# 22I-0813
# CS-E

##### Simple Reflex Agents
##### Task 1: Marks: 5
##### Design a simple reflex agent, that waters plants efficiently in a greenhouse.
Task Description:
###### Environment:
- A 3x3 grid where each cell contains plants (Dry/Wet).
###### Percepts:
- The agent senses the soil moisture level of the cell it is currently in.
###### Actions:
- Move in four predefined directions: Up → Right → Down → Left.
- Water the plant in its current cell if the soil is Dry.
###### Goal.
- Make each plant Wet.

In [36]:
import random
import numpy as np

class Agent:
    def __init__(self, grid):
        self.grid = grid
        self.dry_count = self.count_dry_cells()
        self.current_position = (0, 0)
        print("Dry Count:", self.dry_count)
        self.water_plants()

    def count_dry_cells(self):
        return np.sum(self.grid == 0)

    def water_plants(self):
        while self.dry_count:
            # Water the plant if the current cell is dry
            if self.grid[self.current_position] == 0:
                self.grid[self.current_position] = 1
                self.dry_count -= 1
                print("Dry cells remaining:", self.dry_count)
                print(self.grid)

            # Move the agent to the next position
            self.move_agent()

        print("All plants are wet!")
        print(self.grid)
        return self.grid

    def move_agent(self):
        #  up, right, down, left
        directions = [(0, 1), (1, 0), (0, -1), (-1, 0)]

        # Check for adjacent dry cells and move there if possible
        for direction in directions:
            new_position = (self.current_position[0] + direction[0],self.current_position[1] + direction[1])
            if 0 <= new_position[0] < 3 and 0 <= new_position[1] < 3 and self.grid[new_position] == 0:
                self.current_position = new_position
                return

        # pick random direction
        while True:
            random_direction = random.choice(directions)
            new_position = (self.current_position[0] + random_direction[0],self.current_position[1] + random_direction[1])
            if 0 <= new_position[0] < 3 and 0 <= new_position[1] < 3:
                self.current_position = new_position
                return

grid = np.random.randint(2, size=(3, 3))
agent = Agent(grid)

Dry Count: 7
Dry cells remaining: 6
[[1 1 0]
 [1 0 0]
 [0 0 0]]
Dry cells remaining: 5
[[1 1 0]
 [1 1 0]
 [0 0 0]]
Dry cells remaining: 4
[[1 1 0]
 [1 1 1]
 [0 0 0]]
Dry cells remaining: 3
[[1 1 0]
 [1 1 1]
 [0 0 1]]
Dry cells remaining: 2
[[1 1 0]
 [1 1 1]
 [0 1 1]]
Dry cells remaining: 1
[[1 1 0]
 [1 1 1]
 [1 1 1]]
Dry cells remaining: 0
[[1 1 1]
 [1 1 1]
 [1 1 1]]
All plants are wet!
[[1 1 1]
 [1 1 1]
 [1 1 1]]


Task 2:                                                                                                                       Marks: 5

Implement a model-based agent for an autonomous cleaning robot that navigates a 5×5 room,
cleans dirt, and avoids obstacles. The robot maintains an internal model of the environment,
updates its knowledge dynamically, and optimizes its cleaning path.

Task Description:

• Environment:

▪ The warehouse is represented as a 5×5 grid, containing:

• Dirt (D): Locations that need to be cleaned.

• Obstacles (#): Fixed furniture that the robot must avoid.

• Empty Spaces (.): Areas where the robot can move freely.

• Robot (R): The agent navigating the room.

▪ Dynamic Changes in the Environment:

• The robot remembers which locations have been cleaned.

• The internal model updates after each cleaning action.

• Percepts:

▪ The grid layout (Dirt, Obstacles, Free Spaces).

▪ The current location of dirt (dirty or already cleaned).

▪ The robot’s current position in the room.

• Actions:

▪ Move (Up, Down, Left, Right): Navigate through empty spaces.

▪ Clean (C): Remove dirt from the current position.

▪ Update Internal Model: Mark cleaned areas to avoid unnecessary revisits.

• Goal:

▪ Maintain an Internal Model:

• Store the room layout, including dirt, obstacles, and empty spaces.

• Update the model when a dirt spot is cleaned.

▪ Adapt to Changes:

• Avoid revisiting already cleaned areas.

• Optimize movement using BFS for efficiency.

▪ Complete All Deliveries:

• Navigate to all dirt locations using the shortest path.

• Clean all dirty spots and update the internal model dynamically.

In [50]:
from collections import deque

class CleaningRobot:
    def __init__(self, grid, start_position):
        self.grid = grid
        self.position = start_position
        self.cleaned = set()  #cleaned locations track
        self.directions = [(-1, 0), (1, 0), (0, -1), (0, 1)] # Up, Down, Left, Right

    def is_valid_move(self, x, y):
        #check if move in bounds, not an obstacle, and not already cleaned
        return 0 <= x < 5 and 0 <= y < 5 and self.grid[x][y] != '#' and (x, y) not in self.cleaned

    def bfs(self, start, goal):
        # bfs for shortest path to goal
        queue = deque([(start, [])]) 
        visited = set([start]) 
        
        while queue:
            # dequeue next position and its path
            current, path = queue.popleft()  
            if current == goal:
                print(start,path)
                return path  
            
            #check all neighbors
            for dx, dy in self.directions:
                nx, ny = current[0] + dx, current[1] + dy
                if self.is_valid_move(nx, ny) and (nx, ny) not in visited:
                    # add valid neighbors to queue
                    visited.add((nx, ny)) 
                    queue.append(((nx, ny), path + [(nx, ny)]))  
        print("no path")
        return []  

    def clean(self, x, y):
        if self.grid[x][y] == 'D':
            print(f"Cleaning dirt at{x}, {y})")
            self.grid[x][y] = '.'  
            self.cleaned.add((x, y))  

    def navigate_and_clean(self):
        # Store all dirty locations
        dirt_locations = [(i, j) for i in range(5) for j in range(5) if self.grid[i][j] == 'D']

        while dirt_locations:
            current_dirt = dirt_locations[0]
            path = self.bfs(self.position, current_dirt)  # bfs path to dirt
            if path:
                for (x, y) in path:  # Follow the path to dirt
                    self.position = (x, y)  
                    print(f"Moving to ({x}, {y})")
                
                # Clean the current dirt
                self.clean(x, y)
                dirt_locations.pop(0) 

        print("All dirt cleaned!")  

grid = [
    ['.', '.', 'D', '.', '#'],
    ['#', '.', '#', '.', 'D'],
    ['.', '#', '#', '#', '.'],
    ['.', '.', 'D', '.', '.'],
    ['#', '.', '#', '.', '#']
]

start_position = (0, 0)  

print("Initial Grid:")
for row in grid:
    print(' '.join(row))

robot = CleaningRobot(grid, start_position)
robot.navigate_and_clean()

print("Final Grid:")
for row in grid:
    print(' '.join(row))


Initial Grid:
. . D . #
# . # . D
. # # # .
. . D . .
# . # . #
(0, 0) [(0, 1), (0, 2)]
Moving to (0, 1)
Moving to (0, 2)
Cleaning dirt at0, 2)
(0, 2) [(0, 3), (1, 3), (1, 4)]
Moving to (0, 3)
Moving to (1, 3)
Moving to (1, 4)
Cleaning dirt at1, 4)
(1, 4) [(2, 4), (3, 4), (3, 3), (3, 2)]
Moving to (2, 4)
Moving to (3, 4)
Moving to (3, 3)
Moving to (3, 2)
Cleaning dirt at3, 2)
All dirt cleaned!
Final Grid:
. . . . #
# . # . .
. # # # .
. . . . .
# . # . #
